# Examen de statistiques descriptives
(Jérôme Lacaille et Florent Forest)

<p style="color:red; background:yellow">
Vous soumettrez votre réponse sous la forme d'un fichier notebook (MACS3-SD19-Prenom_Nom.ipynb) que vous enverez par courrier électronique avant le 1er décembre aux deux adresses suivantes :
</p>

- jerome.lacaille@safrangroup.com
- forest@lipn-univ-paris13.fr.

<p style="color:red; background:yellow">    
A ce notebook est associé un fichier `hdf5set.py` contenant des codes auxiliaires et un fichier `requirements.txt`obtenu à partir de la commande "pip freeze" qui contient la description de l'environnement de travail permettant d'exécuter ce code.
</p>

<p style="color:red; background:yellow">
L'examen se compose de plusieurs questions optionnelles, chaque question compte pour un nombre de point défini. La note totale sera la somme des points obtenus par votre présence lors des cours et TD (1/2 point par séance, soit 6 points en tout), plus les poinst accumulés par cet examen (20 points), la note finale sera majorée à 20.
</p>

<p style="color:red; background:yellow">
Attention cependant, toute journée de retard compte pour un demi point de moins et une recopie évidente (plagiat) du travail d'un de vos camarade compte pour -4 points par contrevenant.
</p>

<p style="color:red; background:yellow">
Bon travail !
</p>

#### Corrections

<p style='color:red; background:yellow'>
Le tableau ci-dessous servira d'évaluation, vous voyez ainsi comment la notation sera découpée en éléments de base. Surtoutr n'hésitez pas à commenter vos codes et ne laissez pas d'affichages sans légende.
</p>

|Question     |Elément                |Points|Note |
|-------------|-----------------------|------|-----|
|Théorie      |Légende                |+1    |     |
|             |R0 & Remp              |+2    |     |
|             |Robustesse             |+1    |     |
|-------------|-----------------------|------|-----|
|Code         |Interactivité          |+1    |     |
|             |Callbacks              |+1    |     |
|-------------|-----------------------|------|-----|
|Manipulation |Explications           |+2    |     |
|             |Code                   |+2    |     |
|-------------|-----------------------|------|-----|
|Modélisation |Durée + moyennes       |+1    |     |
|             |Consommation           |+1    |     |
|             |Modèle                 |+1    |     |
|             |Robustesse             |+1    |     |
|             |Température            |+1    |     |
|-------------|-----------------------|------|-----|
|Visualisation|Données                |+1    |     |
|             |Affichage              |+2    |     |
|             |Catégories             |+1    |     |
|             |Température            |+1    |     |
|-------------|-----------------------|------|-----|

<p style='color:red; background:yellow'>
Les zones en jaune correspondent au sujet.
</p>

## Première partie, robustesse et surparamétrisation
<p style="color:red; background:yellow">
Dans un premier temps je vais fabriquer un jeu de données artificiel formé d'un bout de sinusoïde. Un tirage de points aléatoires avec un peu de bruit sera notre donnée d'entrée. On cherche à retrouver le modèle sous-jacent (la sinusoïde) par une approche polynomiale. Nous allons jouer sur le degré du polynome.
    </p>

In [1]:
#Indispensable pour la première partie

import os
import numpy as np
from numpy.polynomial.polynomial import polyfit, polyval

import plotly.graph_objs as go
import plotly.express as px

<p style="color:red; background:yellow">
Fabrication d'un jeu de données complet (x,y) et d'un tirage aléatoire de points (x1,y1) bruités.
    </p>

In [2]:
x = np.linspace(-np.pi/4,9*np.pi/4,200)
y = np.sin(x)

sigma = 0.3 # Ecart type du bruit rajouté.
x1 = np.sort(np.random.choice(x,30)) # Je tire 30 points aléatoirement.
y1 = np.sin(x1) + sigma*np.random.randn(len(x1))

<p style="color:red; background:yellow">
J'aime bien `plotly` pour ses possibilités d'interactivité que nous verrons dans la seconde partie, rien ne vous interdit d'utiliser d'autre outils d'affichage.
</p>

In [3]:
data = [go.Scatter(x=x,y=y,line={'color':"green"},name="sinus"),
        go.Scatter(x=x1,y=y1,mode='markers',
                   marker={'symbol':'circle-open','size':8,'color':'blue'},
                   name='obs')]
layout = go.Layout(title="Sinusoïde")
go.Figure(data,layout)

<p style="color:red; background:yellow">
Cette courbe présente le modèle (la sinusoïde) en vert continu et les points tirés sous la forme de ronds bleus.
</p>

<p style="color:red; background:yellow">
Nous allons créer maintenant trois modèles de régressions polynomiales d'ordres 2, 5 et 20.
    </p>

In [4]:
p = polyfit(x1,y1,2)
z2 = polyval(x,p)
p = polyfit(x1,y1,5)
z5 = polyval(x,p)
p = polyfit(x1,y1,20)
z20 = polyval(x,p)

<p style="color:red; background:yellow">
Le code suivant affiche les résultats des régressions.
    </p>

In [5]:
data = [go.Scatter(x=x,y=y,line={'color':"green"},name="sinus"),
        go.Scatter(x=x1,y=y1,mode='markers',
                   marker={'symbol':'circle-open','size':8,'color':'blue'},
                   name='obs'),
        go.Scatter(x=x,y=z2,line={'color':"red",'dash':"dot"},name="deg2"),
        go.Scatter(x=x,y=z5,line={'color':"cyan",'dash':"dot"},name="deg5"),
        go.Scatter(x=x,y=z20,line={'color':"violet",'dash':"dot"},name="deg20"),
       ]
layout = go.Layout(title="Sinusoïde",yaxis={'range':[-1.5,1.5]})
go.Figure(data=data,layout=layout)

### Question théorique (+4)

<p style="color:red; background:yellow">
Pour 4 points, il s'agit de commenter l'image précédente.
N'oubliez pas de donner une légende (1pt). Celle ci-doit décrire ce que l'on voit et ce qu'il faut observer sur cette image.
</p>

_Légende :_

Des observations bruitées (points bleus) sont obtenus à partir d'un modèle de sinusoïde (courbe verte). Trois régressions sont calibrées à partir des points bleus. La courbe rouge pointillée est un modèle polynomial de degré 2, la courbne bleue pointillée est un modèle polynomial de degré 5 et la courbe magenta pointillée est un modèle polynomial de degré 20. On observe que le nombre de paramètres du modèle ne doit pas être trop petit pour réaliser une bonne estimation, mais pas trop gros non plus car sinon on s'éloigne du modèle original. On n'est plus robuste.

<p style="color:red; background:yellow">
Vous vous attacherez à expliquer comment on voit les différentes erreurs (empirique, généralisation) depuis ce type d'image et rappeler les éléments fondamentaux relatifs à la robustesse d'une procédure d'apprentissage. (N'oubliez pas de compléter la légende ci-dessus : faites comme si vous voyez cette image et que vous attendiez une explication des axes et des courbes. Expliquez ce que l'on doit observer.)
<p>

_Réponse_ :

On voit qu'un modèle de faible degré de liberté (2 pour la courbe rouge pointillée) n'est pas assez précis ni pour estimer les points observés, ni le modèle original.

À l'opposé, un modèle trop riche (de degré 20 courbe magenta pointillée) va estimer avec précision les points d'apprentissage mais pas le modèle de base. Dans ce cas on parle de sur-paramétrisation.

On note "erreur empirique" l'écart entre les données d'apprentissage et la prédiction. On note aussi "erreur de généralisation" l'écart entre le modèle sous-jacent (la vérité) et la prédiction. Ce qui nous intéresse c'est une optimisation du degré de liberté qui minimise l'erreur de généralisation.

Dans notre cas l'erreur de généralisation peut se calculer assez facilement en prennant un échantillonnage assez large d'abscisses (ce n'est pas trop grave si on risque d'avoir parmi les points choisis des points d'apprentissage si ceux-ci ne sont pas trop nombreux.)

Le code suivant va identifier un degré de polynome optimal. On sait d'avance que ce degré est inférieur à 20 gràce à notre observation graphique.

In [6]:
from numpy.linalg import norm

N=15
R = np.zeros(N)
for k in range(0,N):
    pk = polyfit(x1,y1,k)
    z = polyval(x,pk)
    R[k] = norm(z-y)

data = [go.Scatter(x=np.arange(0,N),y=R)]      
layout = go.Layout(title="Erreur de généralisation",
                  xaxis_title="degré du polynôme",
                  yaxis_title="erreur L2")
go.Figure(data=data,layout=layout)

_Cette courbe montre l'évolution de l'erreur de généralisation en fonction du degré du polynôme._

In [7]:
print("Le minimum d'erreur L2 ({:.2f}) \
est obtenu pour le degré {}."
      .format(R.min(),R.argmin()))

Le minimum d'erreur L2 (1.62) est obtenu pour le degré 5.


#### Remarques du correcteur

<p style='color:red; background:yellow'>
Dans cette réponse j'attends une définition du risque empirique, du risque de généralisation et des arguments qui expliquent leurs valeurs respectives en fonction de la complexité du modèle choisi pour l'estimation. J'attends aussi une méthode permettant de minimiser l'erreur de généralisation et dans ce cas précis d'identifier un degré de polynôme optimal. 
</p>

## Seconde partie, analyse de données

<p style='color:red; background:yellow'>
Les données de cet exercice sont stockées dans un fichier HDF5. Chaque observation est un vol d'un avion.
</p>

In [8]:
# Indispensable pour la seconde partie.

import pandas as pd
from tabata import Opset

<p style='color:red; background:yellow'>
    Les codes du fichier auxiliaire "hdf5set" vous permettent de manipuler des listes de DataFrames pandas stockées dans un fichier HDF5.
    </p>

In [45]:
storename = "../data/in/AFL1EB.h5"
S = Opset(storename)
n=0
for df in S:
    t0 = df.index[0]
    print("{:2d} : {:10s}  --> {}".format(n,df.index.name,t0))
    n = n+1

 0 : record_00   --> 2012-07-10 11:08:00
 1 : record_01   --> 2012-07-10 14:10:00
 2 : record_02   --> 2012-07-27 03:40:00
 3 : record_03   --> 2012-07-27 06:34:00
 4 : record_04   --> 2012-08-03 01:24:00
 5 : record_05   --> 2012-08-03 03:25:00
 6 : record_06   --> 2012-12-28 09:31:00
 7 : record_07   --> 2013-01-05 06:26:00
 8 : record_08   --> 2013-01-05 09:43:00
 9 : record_09   --> 2013-01-05 10:40:00
10 : record_10   --> 2013-01-13 08:13:00
11 : record_11   --> 2013-01-19 20:25:00
12 : record_12   --> 2013-01-29 03:58:00
13 : record_13   --> 2013-01-29 06:38:00
14 : record_14   --> 2013-02-06 02:49:00
15 : record_15   --> 2013-02-06 06:10:00
16 : record_16   --> 2013-02-06 10:26:00
17 : record_17   --> 2013-02-06 13:10:00
18 : record_18   --> 2013-02-06 19:48:00
19 : record_19   --> 2013-02-06 22:48:00
20 : record_20   --> 2013-02-07 04:40:00
21 : record_21   --> 2013-02-07 07:05:00
22 : record_22   --> 2013-02-07 10:34:00
23 : record_23   --> 2013-02-07 14:02:00
24 : record_24  

<p style='color:red; background:yellow'>
    On dispose de 52 enregistrements. Il y a 6 variables par enregistrement. Les noms des variables sont formés du nom suivi de l'unité entre crochets.
</p>

In [46]:
df.columns

Index(['ALT[m]', 'Tisa[K]', 'TAS[m/s]', 'Vz[m/s]', 'Masse[kg]', 'F[N]'], dtype='object')

<div style='color:red; background:yellow'>
    
* ALT[m] : l'altitude de l'avion en mètres.
* Tisa[K] : la température standard en Kelvin.
* TAS[m/s] : La vitesse air de l'avion (Total Air Speed) en mètre par seconde.
* Vz[m/s] : la vitesse de montée en mètre par seconde.
* Masse[kg] : la masse de l'avion en kilogramme.
* F[N] : la poussée en Newton.

<p style='color:red; background:yellow'>
    Pour récupérer un enregistrement spécifique il est possible d'adresser directement la fonction "read_hdf" de pandas.
    </p>

In [47]:
df = pd.read_hdf(storename,"record_24")
df

,ALT[m],Tisa[K],TAS[m/s],Vz[m/s],Masse[kg],F[N]
record_24,,,,,,
2013-02-08 04:30:00,-11.411712,830.077411,0.0,-3.665044e-02,15460.643012,0.0
2013-02-08 04:30:01,-11.411712,830.077411,0.0,-3.596550e-02,15460.643012,0.0
2013-02-08 04:30:02,-11.411712,830.077411,0.0,-3.528057e-02,15460.643012,0.0
2013-02-08 04:30:03,-11.411712,830.077411,0.0,-3.459563e-02,15460.643012,0.0
2013-02-08 04:30:04,-11.411712,830.077411,0.0,-3.391069e-02,15460.643012,0.0
...,...,...,...,...,...,...
2013-02-08 06:59:10,98.901504,828.091773,0.0,-6.562746e-15,13814.603910,0.0
2013-02-08 06:59:11,98.901504,828.091773,0.0,-6.806036e-15,13814.603910,0.0
2013-02-08 06:59:12,98.901504,828.091773,0.0,-7.049326e-15,13814.603910,0.0


<p style='color:red; background:yellow'>
    Ou bien directement indexer l'Opset.
</p>

In [48]:
S[24]

,ALT[m],Tisa[K],TAS[m/s],Vz[m/s],Masse[kg],F[N]
record_24,,,,,,
2013-02-08 04:30:00,-11.411712,830.077411,0.0,-3.665044e-02,15460.643012,0.0
2013-02-08 04:30:01,-11.411712,830.077411,0.0,-3.596550e-02,15460.643012,0.0
2013-02-08 04:30:02,-11.411712,830.077411,0.0,-3.528057e-02,15460.643012,0.0
2013-02-08 04:30:03,-11.411712,830.077411,0.0,-3.459563e-02,15460.643012,0.0
2013-02-08 04:30:04,-11.411712,830.077411,0.0,-3.391069e-02,15460.643012,0.0
...,...,...,...,...,...,...
2013-02-08 06:59:10,98.901504,828.091773,0.0,-6.562746e-15,13814.603910,0.0
2013-02-08 06:59:11,98.901504,828.091773,0.0,-6.806036e-15,13814.603910,0.0
2013-02-08 06:59:12,98.901504,828.091773,0.0,-7.049326e-15,13814.603910,0.0


<p style='color:red; background:yellow'>
    La fonction plot du Selector permet de visualiser rapidement le contenu du fichier.
</p>

In [49]:
S.plot()

<p style='color:red; background:yellow'>
    Cette fonction est assez amusante car elle utilise de l'interactivité au sein du notebook jupyter. Jouez un peu avec la liste déroulante, les deux boutons et la barre de défilement. Essayez aussi de cliquer sur la courbe.
</p>

### Question code (+2)

<p style='color:red; background:yellow'>
    Pour deux points supplémentaires, commentez le code de la classe "Selector". Que peut-on faire avec ? Comment fonctionne l'interactivité des boutons "Previous" et "Next", Que se passe-t-il si vous cliquez sur la courbe ? Comment peut-on récupérer les points selectionnés ? Pourquoi la barre de défilement est-elle verticale et à droite de l'écran ?
</p>

<ul style='color:red; background:yellow'>
<li> Décrivez le fonctionnement des callbacks dans le code.
<li> Expliquez pourquoi on a besoin de HBox et VBox.
<li> N'oubliez pas d'expliquer comment on peut récupérer des instants sélectionnées sur les courbes.
</ul>

_Réponse :_

#### Gestion des signaux multivariés
La classe `Opset` permet de gérer des données stockées sous la forme d'une liste d'observations temporelles. Chaque observation est un signal multivarié indexé par le temps. Cette liste doit être stockée dans un fichier au format HDF5 où chaque enregistrement est une observations.

* Les observations sont lues dans l'ordre alphabétique, par un itérateur dérivé de l'`Opset` ou par appel à la méthode `.iterator()`.
* Chaque signal (observation) est synchrones : toutes les colonnes ont la même longuer et le signal est stocké sous la forme d'un DataFrame pandas (`df`).
* Le nom du signal est stocké dans le nom de l'index (`df.index.name`), ce qui permet de nommer les enregistrements indépendamment de leur nom.
* Le nom de chaque colonne correspond au nom de la variable suivi entre crochets de son unité.

#### Affichage des données.
La méthode `.plot()` sert à afficher les données de manière interactive à laide de Plotly au sein d'un notebook Python. Pour cela l'import du package hdf5set exécute la fonction `init_notebook_mode` de Plotly permettant d'interagir au sein du notebook à l'aide de `iplot()`.

Pour naviguer entre les signaux, le Selector utilise des gadgets (boutons, scrollbar, menu) pour passer d'un enregistrement à un autre. Ces gadgets sont issus du packae ipywidgets qui est compatible avec Plotly. Le code de la fonction `.plot()` est un exemple ssez parlant de l'exploitation de gadgets interactifs au sein d'un notebook Jupyter.

* Un menu permet de sélectionner une colonne à afficher.
* Deux boutons (Previous et Next) passent d'un signal au suivant dans l'ordre des enregistrements.
* Une scrollbar verticale sur la droite rappelle l'enregistrement en cours de visualisation et permet aussi de se déplacer aléatoirement entre les enregistrements.

La liste ordonnée des enregistrements est sauvegardée dans une variables `.records`, le signal en cours d'affichage est stocké dans `.sigpos` (en commençant par le n°1), et le nom de la variable à afficher est conservé dans `.colname`.

#### Fonctionnement de l'interactivité
L'interactivité est donnée par l'appel

    out = widgets.interactive(update_plot, colname=wd, sigpos=ws)
  
`wd` est un pointeur vers le menu Dropdown contenat la liste des variables du signal en cours d'affichage et `ws` un pointeur vers la scrollbar.

       wd = widgets.Dropdown(options=df.columns, description="Variable :")
       ws = widgets.IntSlider(value=1, min=1, max=nbmax, step=-1,
                               orientation='vertical',
                               description='Record',
                               layout=widgets.Layout(height='400px'))
                               
Cette dernière est en mode 'vertical' et sa hauteur est fixée à 400 pixels. Le nombre maximum d'enregistrements a été stocké dans la variable `nbmax`. La fonction `interactive` lie le contenu des gadgets au variables locales `colanme` et `sigpos`qui sont passées à la fonction locale `update_plot()` qui fait le travail d'affichage en mettant à jour les éléments définis par le premier affichage initila à l'apple de `.plot()`.

Si la scrollbar apparait sur la droite de l'affichage c'est parce que l'on a juxtaposé deux HBox dans une VBox que l'on revoie comme retour de `.plot()` :

        boxes = widgets.VBox([widgets.HBox([wd, wbp, wbn]), widgets.HBox([
            f, ws])])
        return boxes

La première HBox contient le menu et les deux boutons, et juste en dessous nous plaçons une seconde Hbox qui contient d'abbord la dfigure `f` créée initialement et à sa droite la scrollbar verticale.

    f = go.FigureWidget(data, layout)
    
L'interaction des boutons est assez simple : on le lie à une fonction callback locale `wb_on_click()` que l'on associe à chaque widget par l'appel de `.on_click()`. En fait cette fonction se contente de modifier la valeur de la scrollbar ce qui entrainera automatiquement par interactivité une modification de l'affichage comme si on avait directement cliqué sur la scrollbar.

#### Mise en évidence d'une partie des données
Le Selector bénéficie de deux fonction d'affichage supplémentaire. La première consiste à mettre en évidence par un coloriage en rouge une partie du signal.
Pour cela il faut qu'une variable booléenne soit présente dans le signal multivarié comme une colonne particulière. Le nom de cette variable peut être passé en argument `phase`de `.plot(phase)` et les points 'True' de cette 'phase' seront affichés en rouge.

#### Sélection d'instants
Pour aller plus loin, il faut regarder l'objet `Selector` (du module instants) qui est dérivé de l'`Opset`. Une fonctionnalité originale consiste en la possibilité de sélectionner des instant en cliquant sur la courbe affichée. Une barre verticale va alors apparaitre pour mettre en évidence ces instants graphiquement. On peut sélectionner un seul instant par signal, mais on peut le faire à partir de n'importe quelle variable.
La liste des couples (variable, instant) est stockée dans la variable `sel_instants` du Selector et peut être récupérée par un simple appel.

    S = Selector.plot()
    instants = S.sel_instants
    
L'interactivité de la sélection d'instant est gérée par la fonction callbck locale `selection_fn` qui est ensuite liée à la trace de la courbe bleue par un appel à `.on_click()`.

#### Remarques du correcteur

<p style='color:red; background:yellow'>
Dans cette question vous devez d'abord observer les différentes interactivités de cet affichage et pour chacune d'entre elle expliquer comment elle est programmée par callback. Ensuite il faut comprendre l'organisation des différents éléments à l'écran et expliquer comment on a fait. L'idée est que vous appreniez comment fonctionne ce type d'interactivité (assez simple) dont vous aurez surement besoin dans votre carrière future de data-scientist. Vous pouvez par exemple tester par vous même et identifier les difficultés.
</p>

### Exercice de manipulation (+4)

<div style='color:red; background:yellow'>
Pour 4 points, observez que les données ne sont pas toutes normales, certains enregistrements sont ratés, probablement à cause de décalages horaires ou à cause de problèmes de capteurs. Ne cherchez pas à corriger ces erreurs, mais construisez un nouveau "hdf5set" (fichier HDFS) sans les données anormales.

Expliquez votre démarche et pourquoi vous jugez ces enregisterments anormaux. Si vous n'arrivez pas à construire un automatisme, n'hésitez pas à supprimer les enregistrements qui seront génant par la suite.

In [50]:
# Construction d'un hdf5set sans données anormales.
cleanstore = "../data/out/AFL1EB_C.h5"
badstore = "../data/out/AFL1EB_B.h5"

CS = Opset(cleanstore).clean() # Je détruit l'ancien stockage
BS = Opset(badstore).clean()   # au cas ou le code écrit ne fonctionnerait pas 
 
    
# Construction d'un hdf5set sans données anormales.
for df in S:
    name = df.index.name
    x = df.index
    n = np.sum(df["ALT[m]"]>8000)
    if n>300 and x[1]>x[0]: # On suppose qu'au moins les deux premiers instants sont bons.
        t = (x-x[0]).total_seconds()
        dt = np.diff(t)
        i = np.argwhere(dt != dt[1])
        if len(i)>0:
            df.index = pd.date_range(x[0],periods=len(df),freq=x[1]-x[0])
            df.index.name = name # Ne pas oublier de récupérer le nom.
        CS.put(df)
    else:
        BS.put(df) # On supprime les vols trop bas (plus de 300 mesures au dessus de 8000ft)

In [51]:
print('Le nombre total de vols conservés est ', len(CS))

Le nombre total de vols conservés est  48


<div style='color:red; background:yellow'>
Quel est la taille de ce nouveau jeu de données ? Faites un peu de statistiques descriptives ici, boites à moustaches, moyennes, variances ... 

_Réponse_ :

Il y a plusieurs façon de nettoyer les données, on ne vous demandait pas de tout faire mais je vais présenter une option ici.

Comme j'ai besoin de vérifier graphiquement mes résultats et que je vais surement avoir plusieurs itérations à faire avant de terminer, je vais stocker les signaux à garder dans un fichier HDF5 `cleanstore` et ceux à supprimer dans `badstore`. Pour faciliter mes itérations je commence par supprimer les anciennes versions de ces fichiers.

Ma solution est uniquement basée sur l'observation de l'altitude. Notez aussi que l'exploitation de la poussée était tout aussi bonne car dans ce cas particulier la poussée n'avait pas été calculée. Je ne vais garder que les vols qui passent au dessus de 8000 pieds et y restent 5 minutes.

Pour les données mal gérées du point de vue temporel, je me suis rendu compte que cela était vraisemblablement dû à un changement de fuseau horaire. Hors les données sont toujours régulièrement échantillonnées, en récupérant la date initiale et la fréquence d'acquisition, il m'est ainsi possible de reconstruire mes signaux sans perdre d'information. Par contre comme je modifie l'index il faut que je pense à sauvegarder le nom de l'enregistrement et à le remettre en place dans le nouveau signal.

In [52]:
# Affichage des signaux conservés et réparés.
CS.rewind().plot()

Je m'aperçois que l'enregistrement n°10 (le 7ième conservé) a un atterrissage raté, mais ce n'est pas vraiment génant puisque pour la suite de l'exercice je ne m'intéresserai qu'aux points de croisière (au dessus de 8000 pied) et avant pour le décollage.

In [53]:
# Affichage des données supprimées.
BS.rewind().plot()

On va faire un affichage de résumé par vol en étudiant notamment s'il reste un effet dû à la durée du vol.

In [54]:
mv = pd.DataFrame()
for df in CS:
    dh = (df.index[-1]-df.index[0]).total_seconds()/3600
    cs = (df["Masse[kg]"][-1]-df["Masse[kg]"][0])/dh
    mv = mv.append(pd.DataFrame({"Durée":dh,
                                 "AltMax":max(df["ALT[m]"]),
                                 "TempMin":min(df["Tisa[K]"]),
                                 "VMax":max(df["TAS[m/s]"]),
                                 "Conso":cs},
                           index=[df.index.name]))

In [55]:
fig = px.scatter_matrix(mv,dimensions=["AltMax","TempMin","VMax","Conso"],color="Durée")
fig.show()

Malgré la normalisation par la durée on observe encore un léger effet de celle-ci sur la consommation plus faible des vols très courts. Mais celui-ci est aussi lié à l'altitude maximale et la vitesse. Cela est normal vu que l'on peut supposer que des vols courts montent moins haut. On se dispensera donc de la durée par la suite dans les analyses.

On note aussi une corrélation extrèmement forte entre température et altitude. C'est un effet très connu, il faudra donc choisir entre l'une de ces deux variables.

#### Remarques du correcteur

<p style='color:red; background:yellow'>
L'idée est de comprendre le type d'anomalie et d'écrire un code qui les élimine. Imaginer un jeu de données beaucoup plus grand que celui dont vous disposez, vous n'irez sûrement pas regarder chaque courbe, mais avec un peu de logique il devient facile d'extraire les données qui ne nous conviennent pas.
</p>

### Exercice de modélisation (+5)

Dans un premier temps vous allez extraire des données la phase de croisière. Tous les moyens sont bons, soyez malin, mais il serait judicieux de construire un jeu de données intermédiaire (hdf5set) pour qu'on puisse observer cette extraction graphiquement. L'idéal serait de rajouter à la main dans un `Selector` des instants de début et de fin de croisère.

Ensuite vous crérez un tableau contenant pour chaque vol les résultats des calculs suivants.

* Calculez la durée de chaque croisière.
* Calculez l'altitude moyenne en phase de croisière de chaque vol.
* Calculez la consommation moyenne de carburant pendant la croisière.
* Calculez la température extérieure moyenne pendant la croisière.
* Calculer la vitesse air moyenne pendant la croisière.

Finalement vous construirez un modèle de la consommation moyenne de carburant en fonction des autres données. Vous estimerez l'erreur de généralisation de votre modèle. Quelles sont les variables vraiment importantes ?

_Réponse_ :

La première étape consiste à extraire le début et la fin de croisière. Ensuite, pour vérifier graphiquement notre calcul on va utiliser une astuce implémentée dans le sélecteur en définissant une variable "CR" booléenne qui correspondra à la croisière.

Pour prendre en compte les phases stabilisées autour de changements de palier on va considérer que l'on ne conserve que les zones stables au dessus de 2000 mètres en dessous de l'altitude maximale.

In [56]:
for df in CS:
    mx = max(df["ALT[m]"])
    df["CR"] = (df["ALT[m]"]>mx-2000) & (abs(df["Vz[m/s]"])<1)
    CS.put(df)

In [57]:
CS.plot("CR",pos=0)

Ensuite, pour chaque signal on extrait les informations correspondant à la phase de croisière et on place des données dans un DataFrame pandas `cr`.

In [58]:
cr = pd.DataFrame(columns=["Dt","Alt","Conso","Temp","Speed"])
for df in CS:
    df = df[df["CR"]]
    dw = df["Masse[kg]"][0]-df["Masse[kg]"][-1]
    dt = df.index[-1]-df.index[0]
    dh = dt.total_seconds()/3600
    cr = cr.append(pd.DataFrame({'Dt'      : dt.total_seconds(),
                                 'Alt'     : np.mean(df["ALT[m]"]),
                                 'Conso'   : dw/dh,
                                 'Temp'    : np.mean(df["Tisa[K]"]),
                                 'Speed' : np.mean(df["TAS[m/s]"])},
                                index=[df.index.name]))

In [59]:
cr

,Dt,Alt,Conso,Temp,Speed
record_00,3588.0,10671.133888,765.226270,637.791590,527.918642
record_01,3644.0,10593.096765,756.388550,639.196258,531.647961
record_02,3021.0,11093.924898,712.095081,630.181352,504.782817
record_03,2098.0,11410.850362,689.501282,624.477115,506.824279
record_04,692.0,9192.230074,765.289568,664.411859,497.990777
record_05,384.0,8874.689170,824.170144,670.127595,512.994344
record_10,297.0,11094.692926,639.573517,630.167527,514.429109
record_11,3375.0,10459.989717,765.484859,641.592185,506.980747
record_12,2364.0,11093.934557,717.891309,630.181178,513.211115
record_13,2723.0,10777.193163,721.184414,635.882523,509.797599


On retrouve la relation linéaire claire entre la température et l'altitude.

In [60]:
data = [go.Scatter(x=cr["Alt"],y=cr["Temp"],mode="markers")]     
layout = go.Layout(title="Relation entre altitude et température",
                   xaxis_title="Altitude (m)",
                   yaxis_title="Température (K)")
go.Figure(data,layout)

On peut donc éliminer l'altitude de l'équation et étudier le lien entre consommation, vitesse et température. Bien entendu la durée ne doit pas plus être prise en compte car on a déjà normalisé la consommation par la durée du vol.

On va maintenant établir un modèle simple (linéaire) entre la consommation moyenne et les autres informations (pas la durée qui ne devrait plus intervenir).

In [61]:
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [62]:
mod = smf.ols(formula='Conso ~ Temp + Speed', data=cr)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  Conso   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     47.15
Date:                Thu, 28 Oct 2021   Prob (F-statistic):           9.08e-12
Time:                        15:51:03   Log-Likelihood:                -210.83
No. Observations:                  48   AIC:                             427.7
Df Residuals:                      45   BIC:                             433.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1624.9836    242.997     -6.687      0.0

* Le F-Test (F-statistic) est un test de Fischer basé sur le rapport entre variance expliquée et variance non-expliquée.
* Le test Omnibus vérifie si tous les facteurs sont influents, il complète le F-test mais on peut aussi se rendre compte de l'existence d'un facteur inutile en regardant les p-values de chaque coefficient.
* Le test de Durbin-Watson est utile sur les séries temporelles, il vérifie s'il ne reste pas d'auto-corrélation résiduelle dans les résidus. $DW\in[0;4]$, et 2-DW donne le sens de l'autocorrélation résiduelle des résidus, donc si DW est proche de 2 il n'y a pas d'autocorrélation, si DW est proche de 0 l'autocorrélation est positive et elle est négative si DW est proche de 4.
* Le test de Jarque-Bera teste la normalité des résidus. $JB=\frac{n-k}{6}(S^2+(K-3)^2/4)$ où S est le skewness (asymétrie) et K le Kurtosis (aplatissement si faible).
* Le "condition number" mesure l'influence des entrées sur la sortie : de combien peut chenger la sortie quand on modifie l'entrée. Ici cela n'a pas de sens à cause de l'intercept qui est élevé. Ne regarder Cond. No. que si on a d'abord supprimé les biais.

Les résultats montre une estimation qui ne sera pas parfaite, néanmoins on repère immédiatement l'importance des deux variables Vitesse et Température dont les p-values sont inférieures à $10^{-3}$ et que l'on ne peut donc pas négliger. 

Pour l'affichage il vaut mieux organiser les données par valeurs monotones de la consommation et regarder la dispersion autour de la prédiction.

In [63]:
p, i_d, i_u = wls_prediction_std(res)
z = res.predict()
cr["hConso"] = z
cr["down"] = i_d
cr["up"] = i_u

In [64]:
cr1 = cr.sort_values("Conso")
u = np.arange(1,1+len(cr))
u2 = list(u)+list(u[::-1])
p2 = list(cr1["up"].values)+list(cr1["down"].values[::-1])
data = [go.Scatter(x=u,y=cr1["Conso"],mode="lines+markers",name="obs",hovertext=cr1.index),
        go.Scatter(x=u,y=cr1["hConso"],mode="markers",name="pred"),
        go.Scatter(x=u2,y=p2,fill='toself',showlegend=False)]
layout = go.Layout(title="Prédictions de la consommation",
                   xaxis_title="Vols (réordonnés par consommation croissante)",
                   yaxis_title="Consommation (K)")
go.Figure(data=data,layout=layout)

La toute première mesure semble être bizarre, sinon les consommations observées sont bien de l'odre de grandeur des valeurs attendues.

In [65]:
print("Enregistrement bizarre : ",cr1.index[0])

Enregistrement bizarre :  record_10


In [66]:
u = np.arange(1,1+len(cr))
u2 = list(u)+list(u[::-1])
p2 = list(cr["up"].values)+list(cr["down"].values[::-1])
data = [go.Scatter(x=u,y=cr["Conso"],mode="lines+markers",name="obs",hovertext=cr.index),
        go.Scatter(x=u,y=cr["hConso"],mode="markers",name="pred"),
        go.Scatter(x=u2,y=p2,fill='toself',showlegend=False)]
layout = go.Layout(title="Prédictions de la consommation",
                   xaxis_title="Vols",
                   yaxis_title="Consommation (K)")
go.Figure(data=data,layout=layout)

#### Remarques du correcteur

<p style='color:red; background:yellow'>
La phase de croisière est le temps que passe l'avion en altitude avec une trajectoire à peu près stabilisée. Fabriquez un indicateur qui repère cette phase et construisez un moyen graphique pour que l'on puisse vérifier de la pertinence de l'indicateur. Vous pouvez par exemple vous inspier du Selector. Vous noterez que j'ai rajouté une fonctionnalité au sélector : quand on ajoute le nom d'une colonne booléenne à la fonction plot(), cette colonne est utilisée pour mettre en rouge la partie de courbe où le booléen est "True".
</p>
<p style='color:red; background:yellow'>
    Ensuite vous allez construire un DataFrame contenant les calculs demandés : une ligne sera un vol et les colonnes représentent les résultats des calculs. Faites particulièrement attention à la consommation, si on vous demande un modèle ce n'est pas pour s'aperçevoir que l'on consomme plus quand on vole longtemps, on n'a pas besoin d'analyse de données pour cela. Bien sûr on s'intéresse à l'effet de l'altitude, de la vitesse et de la température sur la consommation moyenne.
</p>
<p style='color:red; background:yellow'>
On vous demande un calcul de robustesse d'un modèle, cela sous-entend qu'il faut estimer la consommation moyenne, vérifier quells variables sont importantes soit par un test statistique sur leurs paramètres d'interaction, soit par une analyse d'importance, puis tester par une méthode adéquate la généralisation du modèle ainsi calculé.
</p>

### Exercice de visualisation (+5)

<div style='color:red; background:yellow'>
Pendant la phase de montée de l'avion on voudrait visualiser la relation entre poussée et vitesse de montée. Préparer un jeu de données et proposez un affichage intelligent. Existe-t-il des catégorie de montées ? La température extérieure joue-elle un rôle ?

_Réponse_ :

On va commencer par extraire les phases de montée comme on l'avait fait précédemment pour la croisière. L'idée est de créer une variable booléenne CL (Climb) correspondant à la montée ce qui permettra d'avoir un affichage de visualisation.

Pour identifier la montée, il suffit d'étudier la "vitesse de montée" et de s'assurer qu'elle est bien positive (mettons > 1 m/s), bien sur il faudra aussi vérifier que la montée doit s'arréter avant le début de la croisière car il peut y avoir des changements de niveau positifs pendant la croisière ou la descente.

Ensuite on fera un tableau de données descriptives de la montée en étudiant la variation d'altitude, la vitesse air moyenne, les vitesses linimales et maximales de montée, la température moyenne. Ce tableau servira à vérifier s'il existe des catégories de montées.

In [67]:
for df in CS:
    t0_cl = min(df[df["Vz[m/s]"]>1].index)
    t0_cr = min(df[df["CR"]].index)
    df["CL"] = (df.index>=t0_cl) & (df.index<t0_cr)
    CS.put(df)

In [68]:
CS.plot("CL",0)

On repère très vite qu'il peut y avoir des paliers durant la montée, cela doit sans doute être discriminant aussi on ajoute une colonne correspondant à la durée passée en deça de 10 m/s.

In [69]:
cl = pd.DataFrame(columns=["Dt","DAlt","VMax","VzMin","VzMax","DtVzMin","Temp"])
for df in CS:
    df = df[df["CL"]]
    da = df["ALT[m]"][-1]-df["ALT[m]"][0]
    vzmin = np.min(df["Vz[m/s]"])
    dt = df.index[-1]-df.index[0]
    dh = dt.total_seconds()/3600
    cl = cl.append(pd.DataFrame({'Dt'      : dt.total_seconds(),
                                 'DAlt'    : da,
                                 'VMax'    : np.max(df["TAS[m/s]"]),
                                 'VzMin'   : np.min(df["Vz[m/s]"]),
                                 'VzMax'   : np.max(df["Vz[m/s]"]),
                                 'DtVzMin' : len(df[df["Vz[m/s]"]<10]),
                                 'Temp'    : np.mean(df["Tisa[K]"])},
                                index=[df.index.name]))

In [70]:
cl

,Dt,DAlt,VMax,VzMin,VzMax,DtVzMin,Temp
record_00,1093.0,10772.656128,522.240575,1.027213,36.514008,53,720.745512
record_01,1120.0,10721.937408,539.716894,1.015753,38.260894,58,723.910437
record_02,1208.0,11102.327808,525.326872,-0.292743,33.774580,194,714.962309
record_03,1131.0,11419.319808,521.145231,1.025182,41.080399,91,712.898222
record_04,906.0,9191.500032,498.221244,1.076826,42.546437,120,744.753663
record_05,937.0,8882.115840,513.770746,-0.169727,44.821944,206,740.822810
record_10,820.0,11089.648128,515.811751,1.052455,54.389634,135,712.098128
record_11,1020.0,10468.343808,513.590870,1.083934,33.777627,58,724.451722
record_12,1140.0,11098.523904,524.424057,1.029824,39.906669,79,714.224550
record_13,984.0,10772.656128,512.140312,1.108740,40.633596,45,720.684160


D'après le sujet il faut étudier l'impact de la température, mais comme il s'agit d'un sujet graphique, une méthode sympatique est de faire une anayse en composantes principale.

In [71]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [72]:
X = StandardScaler().fit_transform(cl.values)
pca = PCA(n_components=3)
pca = pca.fit(X)

In [73]:
total_variance3 = np.sum(pca.explained_variance_ratio_)*100
print("Variance totale expliquée par 3 composantes : {0:.1f} %".format(total_variance3))

Variance totale expliquée par 3 composantes : 81.8 %


In [74]:
import ipywidgets as widgets

Beaucoup plus facile avec des widgets que la méthode des subplots de plotly !

In [75]:
def PCA_FigureWidget(comp1=1,comp2=2):
    pc1 = pca.components_[comp1-1]
    pc2 = pca.components_[comp2-1]
    
    scalex = np.sqrt(pca.explained_variance_[comp1-1])
    scaley = np.sqrt(pca.explained_variance_[comp2-1])
    
    data = [go.Scatter(x = pc1*scalex, y = pc2*scaley, mode="markers+text", text=cl.columns, textposition="top right",
                       marker=dict(color="red", size=10), showlegend=False, hoverinfo='skip')]
    data2 = [go.Scatter(x=[0,pc1[i]*scalex],y=[0,pc2[i]*scaley],mode="lines",
                        line=dict(color="red",width=1,dash="dot"),
                        showlegend=False) for i in range(0,cl.shape[1])]
    
    shapes=[go.layout.Shape(type="circle",xref="x",yref="y",x0=-1,y0=-1,x1=1,y1=1,line_color="LightBlue")]

    total_variance2 = pca.explained_variance_ratio_[comp1-1] + pca.explained_variance_ratio_[comp2-1]
    layout= go.Layout(title="Projection dans le plan PC{}xPC{} ({:.1f}%)".format(comp1,comp2,total_variance2*100),
                      xaxis_title="PC{} ({:.1f}%)".format(comp1,pca.explained_variance_ratio_[comp1-1]*100),
                      yaxis_title="PC{} ({:.1f}%)".format(comp2,pca.explained_variance_ratio_[comp2-1]*100),
                      shapes=shapes,
                      xaxis=dict(range=[-1,1]),
                      yaxis=dict(range=[-1,1],scaleanchor="x",scaleratio=1))
    return go.FigureWidget(data+data2,layout)

In [76]:
f12 = PCA_FigureWidget(1,2)
f13 = PCA_FigureWidget(1,3)
widgets.HBox([f12,f13])

    'data': [{'hoverinfo': 'skip',
              'marker': {'color': 'red', 'siz…

On s'aperçoit que la température est importante.
On recheche une catégorisation de toutes ces données. On utilise les KMeans pour trouver des catégories de montées. On rechreche trois clusters.

In [77]:
from sklearn.cluster import KMeans

labels = KMeans(n_clusters=4, random_state=0).fit_predict(X)

Finalement on affiche chacun de ces clusters dans une projection de l'ACT.

In [78]:
Y = pca.transform(X)
cmap = ["red","green","blue","magenta"]
def PCA_PlotWidget(comp1,comp2):
    scalex = np.sqrt(pca.explained_variance_[comp1-1])
    scaley = np.sqrt(pca.explained_variance_[comp2-1])
    
    data = [go.Scatter(x=Y[:,comp1-1]*scalex,y=Y[:,comp2-1]*scaley,text=cl.index,mode="markers", 
                       marker=dict(size=10,color=labels,colorscale=cmap))]
    
    total_variance2 = pca.explained_variance_ratio_[comp1-1] + pca.explained_variance_ratio_[comp2-1]
    layout= go.Layout(title="Projection dans le plan PC{}xPC{} ({:.1f}%)".format(comp1,comp2,total_variance2*100),
                      xaxis_title="PC{} ({:.1f}%)".format(comp1,pca.explained_variance_ratio_[comp1-1]*100),
                      yaxis_title="PC{} ({:.1f}%)".format(comp2,pca.explained_variance_ratio_[comp2-1]*100))
    return go.FigureWidget(data,layout)

In [79]:
p12 = PCA_PlotWidget(1,2)
p13 = PCA_PlotWidget(1,3)
widgets.HBox([p12,p13])

    'data': [{'marker': {'color': array([0, 0, 3, 1, 2, 2, 1, 0, 1, 0, 3, 0, 0, …

En passant la sourie sur chaque point on peut voir apparaitre le numéro du vol, cela permet d'interpréter la classification.

* Le cluster rouge semble être un cluster médian.
* Le cluster violet correspond à des paliers plus longs (on peut vérifier sur le graphe interactif des vols).
* Le cluster bleu présente un effet sur la température, mais peut-être aussi sur la vitesse maximale ou la durée du vol qui serait plus petite.
* Le cluster vert est peut être lié à des altitudes différentes.**

In [80]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2, 
    subplot_titles=cmap,
    horizontal_spacing=0.01,
    vertical_spacing=0.1,
    shared_xaxes=True, shared_yaxes=True)

i=0
for df in CS:
    df = df[df["CL"]]
    row,col = np.divmod(labels[i],2)
    fig.add_trace(go.Scatter(x=list(range(0,len(df))), y=df["ALT[m]"], 
                             mode="lines", showlegend=False,
                             line=dict(color=cmap[labels[i]])), 
                  row=int(row+1), col=int(col+1))
    i = i+1
fig.update_yaxes(row=1,col=1,title='Alt[m]')
fig.update_yaxes(row=2,col=1,title='Alt[m]')
fig.update_xaxes(row=2,col=1,title='Durée[s]')
fig.update_xaxes(row=2,col=2,title='Durée[s]')

fig.show()
    

Un des intérêts des subplots est la possibilité de lier les échèles horizontals et verticales, ce qui permet de mieux se rendre compte des variations de durée et d'amplitude.

#### Remarques du correcteur

<p style='color:red; background:yellow'>
Il s'agit cette fois d'un exercice de visualisation. Il faut donc montrer à travers un graphique les liaisons entre la poussée, la vitesse, l'altitude et la température mais cette fois pendant la phase de montée de l'avion.
</p>
<p style='color:red; background:yellow'>
    La première étape comme précédemment est de construire un indicateur de phase de montée et d'extraire les données correspondantes. Vous pourrez lisser un peu pour éliminer du bruit et sous-échantillonner votre table de travail. L'effet de la température doit être étudier en premier pour vérifier si on peut la supprimer de l'analyse et ensuite il faut visualiser les données, éventuellement voir si on a une classification possible. Vous noterez que l'altitude est la température sont très liés, mais peut-on supprimer les deux ?
</p>